Natural Language Processing (NLP) is a unique subset of Machine Learning which cares about the real life unstructured data. Although computers cannot identify and process the string inputs, the libraries like NLTK, TextBlob and many others found a way to process string mathematically. Twitter is a platform where most of the people express their feelings towards the current context. In this notebook, we will fetch the data of some individuals from twitter and analyze it to the ability of the indviduals to lead others in many dimensions.


First we will import the libraries that we will be using in this project.

In [1]:
import sys
import os
import time
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
from datetime import datetime, date, time, timedelta
from collections import Counter
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
# to view all columns
pd.set_option("display.max.columns", None)

In [2]:
#Import the necessary methods from tweepy library  

#install tweepy if you don't have it
!pip install tweepy
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#sentiment analysis package
#!pip install textblob
from textblob import TextBlob

#general text pre-processor
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')

#tweet pre-processor 
!pip install tweet-preprocessor
import preprocessor as p

[nltk_data] Downloading package punkt to /Users/DClinton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data=pd.read_csv('\Desktop\export_dataframe.csv')
data

,Rank,Name,Twittername
0,100,Jeffrey Gettleman,@gettleman
1,99,Africa24 Media,@a24media
2,98,Scapegoat,@andiMakinana
3,97,Africa Check,@AfricaCheck
4,96,James Copnall,@JamesCopnall
...,...,...,...
95,5,Julius Sello Malema,@Julius_S_Malema
96,4,News24,@News24
97,3,Jacob G. Zuma,@SAPresident
98,2,Gareth Cliff,@GarethCliff


In [4]:
#striping off the last character in the twittername column
data['Twittername'] = data['Twittername'].str.rstrip(')')

In [5]:
data

,Rank,Name,Twittername
0,100,Jeffrey Gettleman,@gettleman
1,99,Africa24 Media,@a24media
2,98,Scapegoat,@andiMakinana
3,97,Africa Check,@AfricaCheck
4,96,James Copnall,@JamesCopnall
...,...,...,...
95,5,Julius Sello Malema,@Julius_S_Malema
96,4,News24,@News24
97,3,Jacob G. Zuma,@SAPresident
98,2,Gareth Cliff,@GarethCliff


In [6]:
list1=data['Twittername']
list1

0           @gettleman
1            @a24media
2        @andiMakinana
3         @AfricaCheck
4        @JamesCopnall
            ...       
95    @Julius_S_Malema
96             @News24
97        @SAPresident
98        @GarethCliff
99         @Trevornoah
Name: Twittername, Length: 100, dtype: object

In [8]:
'''
from __future__ import unicode_literals
import tweepy
import time
import unicodecsv as csv
import codecs
from importlib import reload
import sys 
with open('your_file.txt', 'r') as targets_file:
    targets_list = targets_file.readlines()

targets_list_filtered = filter(None, targets_list[0].split('\r'))

targets_list_cleaned = [] 

for item in targets_list_filtered:
    targets_list_cleaned.append(item.strip('\n'))

with codecs.open('output.csv', 'w', 'utf-8') as outcsv:
    outfile = csv.DictWriter(outcsv, fieldnames=['uID', 'Username', 'Follower Count', 'Verified'])
    outfile.writeheader()

    for idx, target in enumerate(targets_list_cleaned):
        try:
            user = api.get_user(target)
            outfile.writerow({'uID': target, 'Username': user.name, 'Follower Count': user.followers_count, 'Verified': user.verified})
            print(idx, target, user.name, user.followers_count, user.verified)
        except tweepy.TweepError as e:
            # outfile.writerow(e.message)
            print(idx, target, e.message)
        '''

"\nfrom __future__ import unicode_literals\nimport tweepy\nimport time\nimport unicodecsv as csv\nimport codecs\nfrom importlib import reload\nimport sys \nwith open('your_file.txt', 'r') as targets_file:\n    targets_list = targets_file.readlines()\n\ntargets_list_filtered = filter(None, targets_list[0].split('\r'))\n\ntargets_list_cleaned = [] \n\nfor item in targets_list_filtered:\n    targets_list_cleaned.append(item.strip('\n'))\n\nwith codecs.open('output.csv', 'w', 'utf-8') as outcsv:\n    outfile = csv.DictWriter(outcsv, fieldnames=['uID', 'Username', 'Follower Count', 'Verified'])\n    outfile.writeheader()\n\n    for idx, target in enumerate(targets_list_cleaned):\n        try:\n            user = api.get_user(target)\n            outfile.writerow({'uID': target, 'Username': user.name, 'Follower Count': user.followers_count, 'Verified': user.verified})\n            print(idx, target, user.name, user.followers_count, user.verified)\n        except tweepy.TweepError as e:\n    

In [9]:
#This a code for getting the followers count of a certain user
'''import tweepy

auth = tweepy.OAuthHandler('hFHw04mGPdEztZe5CMVBTsq9l','cczG6tshnnIFZbdrSNIP7r22NzaFfUsKIC6cRdaglVfP5GyYaN' )
auth.set_access_token('990262908696387584-HTgPRm2tySXaq7nBHeY1rhhKdV6umNe','5BTPHpjJl7JewU2mwd3qCek716Fyz8Q3AgDhgJIYxQ2hO' )

api = tweepy.API(auth)

follower_count = api.get_user('@andBeyondSafari').followers_count
print(follower_count)'''

"import tweepy\n\nauth = tweepy.OAuthHandler('hFHw04mGPdEztZe5CMVBTsq9l','cczG6tshnnIFZbdrSNIP7r22NzaFfUsKIC6cRdaglVfP5GyYaN' )\nauth.set_access_token('990262908696387584-HTgPRm2tySXaq7nBHeY1rhhKdV6umNe','5BTPHpjJl7JewU2mwd3qCek716Fyz8Q3AgDhgJIYxQ2hO' )\n\napi = tweepy.API(auth)\n\nfollower_count = api.get_user('@andBeyondSafari').followers_count\nprint(follower_count)"

In [10]:
#Using tweepy and codecs we can be able to get the tweets, retweet_count and favorite_count
'''import tweepy
import codecs

auth = tweepy.OAuthHandler('hFHw04mGPdEztZe5CMVBTsq9l','cczG6tshnnIFZbdrSNIP7r22NzaFfUsKIC6cRdaglVfP5GyYaN' )
auth.set_access_token('990262908696387584-HTgPRm2tySXaq7nBHeY1rhhKdV6umNe','5BTPHpjJl7JewU2mwd3qCek716Fyz8Q3AgDhgJIYxQ2hO' )

api = tweepy.API(auth)
results = api.user_timeline(screen_name="@Trevornoah",count=100)
for result in results:
    print('Text: ', ascii(result.text))
    print('User: ', ascii(result.user.screen_name))
    print('# Retweets: ', result.retweet_count.sum())
    print('# Favorites: ', result.favorite_count)'''

'import tweepy\nimport codecs\n\nauth = tweepy.OAuthHandler(\'hFHw04mGPdEztZe5CMVBTsq9l\',\'cczG6tshnnIFZbdrSNIP7r22NzaFfUsKIC6cRdaglVfP5GyYaN\' )\nauth.set_access_token(\'990262908696387584-HTgPRm2tySXaq7nBHeY1rhhKdV6umNe\',\'5BTPHpjJl7JewU2mwd3qCek716Fyz8Q3AgDhgJIYxQ2hO\' )\n\napi = tweepy.API(auth)\nresults = api.user_timeline(screen_name="@Trevornoah",count=100)\nfor result in results:\n    print(\'Text: \', ascii(result.text))\n    print(\'User: \', ascii(result.user.screen_name))\n    print(\'# Retweets: \', result.retweet_count.sum())\n    print(\'# Favorites: \', result.favorite_count)'

In [11]:
#We can also use this code segment that returns a csv file which has the tweets, retweet_count and favorite_count
import tweepy
import csv
import os
import pandas as pd

consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')

def get_tweets(screen_name):

    auth = tweepy.OAuthHandler('hFHw04mGPdEztZe5CMVBTsq9l', 'cczG6tshnnIFZbdrSNIP7r22NzaFfUsKIC6cRdaglVfP5GyYaN')
    auth.set_access_token('990262908696387584-HTgPRm2tySXaq7nBHeY1rhhKdV6umNe', '5BTPHpjJl7JewU2mwd3qCek716Fyz8Q3AgDhgJIYxQ2hO')
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []  

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name, count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet minus one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab. 
    # Limit set to around 3k tweets, can be edited to preferred number.
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id arg to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200, max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))   

    #transform the tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.id_str, tweet.created_at,tweet.retweet_count,tweet.favorite_count, tweet.text.encode("utf-8")] for tweet in alltweets]

    #write the csv  
    with open('%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","retweet_count","favorite_count","text"])
        writer.writerows(outtweets)

    pass

In [12]:
#Reading our csv files into dataframes
dataframe_1=pd.read_csv("Africaninfluencers - govt_influencers.csv")
dataframe_2=pd.read_csv("\Desktop\export_dataframe.csv")


In [13]:
#getting the the columns with twitter handles
dataframe1=dataframe_1['Twitter_name']
dataframe2=dataframe_2['Twittername']

In [14]:
# fetch the unique handles from the top_100 and leaders dataframes
# convert to list the merge them to be one list.
#we also pass tweepy error 
'''the_100 = dataframe1.unique()
the_leaders_response = dataframe2.unique()
l1 = the_100.astype(str).tolist() 
l2 = the_leaders_response.astype(str).tolist()
accounts = l1 + l2
try:
    if __name__ == '__main__':
    #loop through the handles in the list
        for i,name in enumerate(accounts):
              get_tweets(name)
            
except tweepy.error.TweepError:
    pass
      '''

"the_100 = dataframe1.unique()\nthe_leaders_response = dataframe2.unique()\nl1 = the_100.astype(str).tolist() \nl2 = the_leaders_response.astype(str).tolist()\naccounts = l1 + l2\ntry:\n    if __name__ == '__main__':\n    #loop through the handles in the list\n        for i,name in enumerate(accounts):\n              get_tweets(name)\n            \nexcept tweepy.error.TweepError:\n    pass\n      "

This is a python code(python classes and functions) that will illustrate and assist in searching and fetching data from twitter.

In [15]:
class tweetsearch():
    '''
    This is a basic class to search and download twitter data.
    You can build up on it to extend the functionalities for more 
    sophisticated analysis
    '''
    def __init__(self, cols=None,auth=None):
        #
        if not cols is None:
            self.cols = cols
        else:
            self.cols = ['id', 'created_at', 'source', 'original_text','clean_text', 
                    'sentiment','polarity','subjectivity', 'lang',
                    'favorite_count', 'retweet_count','likes' 'original_author',   
                    'possibly_sensitive', 'hashtags',
                    'user_mentions', 'place', 'place_coord_boundaries']
            
        if auth is None:
            
            #Variables that contains the user credentials to access Twitter API 
            consumer_key = os.environ.get('consumer_key')
            consumer_secret = os.environ.get('consumer_secret')
            access_token = os.environ.get('access_token')
            access_token_secret = os.environ.get('access_token_secret')
            


            #This handles Twitter authetification and the connection to Twitter Streaming API
            auth = OAuthHandler('xxx', 'xxx')
            auth.set_access_token('xxx', 'xxx')
            

        #            
        self.auth = auth
        self.api = tweepy.API(auth) 
        self.filtered_tweet = ''
            

    def clean_tweets(self, twitter_text):

        #use pre processor
        tweet = p.clean(twitter_text)

         #HappyEmoticons
        emoticons_happy = set([
            ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
            ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
            '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
            'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
            '<3'
            ])

        # Sad Emoticons
        emoticons_sad = set([
            ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
            ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
            ':c', ':{', '>:\\', ';('
            ])

        #Emoji patterns
        emoji_pattern = re.compile("["
                 u"\U0001F600-\U0001F64F"  # emoticons
                 u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                 u"\U0001F680-\U0001F6FF"  # transport & map symbols
                 u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                 u"\U00002702-\U000027B0"
                 u"\U000024C2-\U0001F251"
                 "]+", flags=re.UNICODE)

        #combine sad and happy emoticons
        emoticons = emoticons_happy.union(emoticons_sad)

        stop_words = set(stopwords.words('english'))
        word_tokens = nltk.word_tokenize(tweet)
        #after tweepy preprocessing the colon symbol left remain after      
        #removing mentions
        tweet = re.sub(r':', '', tweet)
        tweet = re.sub(r'‚Ä¶', '', tweet)

        #replace consecutive non-ASCII characters with a space
        tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)

        #remove emojis from tweet
        tweet = emoji_pattern.sub(r'', tweet)

        #filter using NLTK library append it to a string
        filtered_tweet = [w for w in word_tokens if not w in stop_words]

        #looping through conditions
        filtered_tweet = []    
        for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
            if w not in stop_words and w not in emoticons and w not in string.punctuation:
                filtered_tweet.append(w)

        return ' '.join(filtered_tweet)            

    def get_tweets(self, keyword, csvfile=None):
        
        
        df = pd.DataFrame(columns=self.cols)
        
        if not csvfile is None:
            #If the file exists, then read the existing data from the CSV file.
            if os.path.exists(csvfile):
                df = pd.read_csv(csvfile, header=0)
            

        #page attribute in tweepy.cursor and iteration
        for page in tweepy.Cursor(self.api.search, q=keyword,count=500, include_rts=False).pages():

            # the you receive from the Twitter API is in a JSON format and has quite an amount of information attached
            for status in page:
                
                new_entry = []
                status = status._json
                
                #filter by language
                if status['lang'] != 'en':
                    continue

                
                #if this tweet is a retweet update retweet count
                if status['created_at'] in df['created_at'].values:
                    i = df.loc[df['created_at'] == status['created_at']].index[0]
                    #
                    cond1 = status['favorite_count'] != df.at[i, 'favorite_count']
                    cond2 = status['retweet_count'] != df.at[i, 'retweet_count']
                    if cond1 or cond2:
                        df.at[i, 'favorite_count'] = status['favorite_count']
                        df.at[i, 'retweet_count'] = status['retweet_count']
                    continue

                #calculate sentiment
                filtered_tweet = self.clean_tweets(status['text'])
                blob = TextBlob(filtered_tweet)
                Sentiment = blob.sentiment     
                polarity = Sentiment.polarity
                subjectivity = Sentiment.subjectivity

                new_entry += [status['id'], status['created_at'],
                              status['source'], status['text'], filtered_tweet, 
                              Sentiment,polarity,subjectivity, status['lang'],
                              status['favorite_count'], status['retweet_count']]

                new_entry.append(status['user']['screen_name'])

                try:
                    is_sensitive = status['possibly_sensitive']
                except KeyError:
                    is_sensitive = None

                new_entry.append(is_sensitive)

                hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
                new_entry.append(hashtags) #append the hashtags

                #
                mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
                new_entry.append(mentions) #append the user mentions

                try:
                    xyz = status['place']['bounding_box']['coordinates']
                    coordinates = [coord for loc in xyz for coord in loc]
                except TypeError:
                    coordinates = None
                #
                new_entry.append(coordinates)

                try:
                    location = status['user']['location']
                except TypeError:
                    location = ''
                #
                new_entry.append(location)

                #now append a row to the dataframe
                single_tweet_df = pd.DataFrame([new_entry], columns=self.cols)
                df = df.append(single_tweet_df, ignore_index=True)

        if not csvfile is None:
            #save it to file
            df.to_csv(csvfile, columns=self.cols, index=False, encoding="utf-8")
            
        return df

Now we search twitter and fetch data using the keywords and hashtags

In [18]:


#get file if you have already downloaded what you wanted
df = pd.read_csv(tweets_file, header=0)

##get data on keywords
ts = tweetsearch()
df = ts.get_tweets('@News24')    #you saved the 
df

In [ ]:
#getting the sum of the columns of the returned data to find the total number of user_mentions
df.sum(axis = 0, skipna = True) 

In [ ]:
'''We will now the data of the top influencers from twitter;the screen_name,description,statuses_count,friends_count,
followers_count,Account_age(in days),Average tweets per day, most mentioned twitter users and most 10 used hashtags.
account_list1 is the list of the twitter handles of the top influencers.'''
try:
    if len(account_list1) > 0:
        for target in account_list1:
            print("Getting data for " + target)
            item =api.get_user(target)
            print("name: " + item.name)
            print("screen_name: " + item.screen_name)
            print("description: " + item.description)
            print("statuses_count: " + str(item.statuses_count))
            print("friends_count: " + str(item.friends_count))
            print("followers_count: " + str(item.followers_count))
            tweets = item.statuses_count
            account_created_date = item.created_at
            delta = datetime.utcnow() - account_created_date
            account_age_days = delta.days
            print("Account age (in days): " + str(account_age_days))
            if account_age_days > 0:
                print("Average tweets per day: " + "%.2f"%(float(tweets)/float(account_age_days)))
            hashtags = []
            mentions = []
            tweet_count = 0
            end_date = datetime.utcnow() - timedelta(days=100)
            for status in tweepy.Cursor(api.user_timeline,id=target).items():
                tweet_count += 1
                if hasattr(status, "entities"):
                    entities = status.entities
                    if "hashtags" in entities:
                        for ent in entities["hashtags"]:
                            if ent is not None:
                                if "text" in ent:
                                    hashtag = ent["text"]
                                    if hashtag is not None:
                                        hashtags.append(hashtag)
                    if "user_mentions" in entities:
                        for ent in entities["user_mentions"]:
                            if ent is not None:
                                if "screen_name" in ent:
                                    name = ent["screen_name"]
                                    if name is not None:
                                        mentions.append(name)
                if status.created_at < end_date:
                    break
            print
            print("Most mentioned Twitter users:")
            for item, count in Counter(mentions).most_common(10):
                print(item + "\t" + str(count))

            print
            print("Most used hashtags:")
            for item, count in Counter(hashtags).most_common(10):
                print(item + "\t" + str(count))

            print
            print ("All done. Processed " + str(tweet_count) + " tweets.")
            print
        
except tweepy.error.TweepError:
    pass  